# Walker

In [ ]:
'''
Walker built on Pymunk physics engine
Single shot EBH learning
E.C. Wong UCSD
'''

# Imports

In [1]:
%matplotlib widget
import numpy as np
import pymunk
from pymunk.vec2d import Vec2d
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
import pickle
import math
import sparse_weights as sw

Loading chipmunk for Darwin (64bit) [/Users/ecwong/miniconda3/lib/python3.6/site-packages/pymunk/libchipmunk.dylib]


# Walker Class

In [2]:
'''
Walker includes a pymunk space and a Walker dude in it
'''

class Walker:
     
    def __init__(self):
        # Create pymunk space
        self.space = pymunk.Space()
        
        # Set walker parameters (MKS system of units)
        # Masses from Ackermann 2007 p.146-7
        self.space.gravity = 0,-9.8
        self.torso_length = 0.80
        self.torso_mass = 45     #38
        self.thigh_length = 0.45
        self.thigh_mass = 10     #7
        self.leg_length = 0.45
        self.leg_mass = 3        #4
        self.foot_length = 0.25
        self.foot_mass = 1
        self.foot_pivot = 0.08 # from heel to ankle joint
        self.rad = 0.03        # not sure what this does 0.02
        self.elas = 0.0        # elasticity
        self.fric = 5.0        # Use huge number to eliminate slipping. Rubber on concrete = 1.0
        self.mid_torso = self.leg_length + self.thigh_length + 0.5 * self.torso_length
        self.total_mass = self.torso_mass + 2 * (self.thigh_mass + self.leg_mass + self.foot_mass)
        self.stability = 0.0
        self.knee_bend = 0.05 * np.pi
        self.stride = 1.0
        self.strike_offset = 0.25 * self.stride
        self.catching_fall = False
        
        # Maximum torques (N-m)
        self.torque_max_hip   = 300
        self.torque_max_knee  = 200
        self.torque_max_ankle = 200 #200
        
        # Calculate scaling for body support using knee torque
        # Torque = M G L sin(bend/2)
        # MGLN is normalized to torque_max_knee
        self.MGLN = self.total_mass * (-self.space.gravity[1]) * 0.5 * (self.leg_length + self.thigh_length) / self.torque_max_knee

        # Vectors for body segments
        self.zero_vec  = Vec2d(0,0)
        self.heel_vec  = Vec2d(-self.foot_pivot,0)
        self.foot_vec  = Vec2d(self.foot_length,0)
        self.leg_vec   = Vec2d(0,self.leg_length)
        self.thigh_vec = Vec2d(0,self.thigh_length)
        self.hip_vec   = self.leg_vec + self.thigh_vec
        self.torso_vec = Vec2d(0,self.torso_length)

        # Add bodies and shapes
        moment = pymunk.moment_for_segment(self.torso_mass,self.zero_vec,self.torso_vec,self.rad)
        self.torso = pymunk.Body(self.torso_mass, moment)
        self.torso.position = self.hip_vec + 0.5 * self.torso_vec
        moment = pymunk.moment_for_segment(self.thigh_mass,self.zero_vec,self.thigh_vec,self.rad)
        self.thighL = pymunk.Body(self.thigh_mass, moment)
        self.thighL.position = self.leg_vec + 0.5 * self.thigh_vec
        self.thighR = pymunk.Body(self.thigh_mass, moment)
        self.thighR.position = self.leg_vec + 0.5 * self.thigh_vec
        moment = pymunk.moment_for_segment(self.leg_mass,self.zero_vec,self.leg_vec,self.rad)
        self.legL = pymunk.Body(self.leg_mass, moment)
        self.legL.position = 0.5 * self.leg_vec
        self.legR = pymunk.Body(self.leg_mass, moment)
        self.legR.position = 0.5 * self.leg_vec
        moment = pymunk.moment_for_segment(self.foot_mass,self.zero_vec,self.foot_vec,self.rad)
        self.footL = pymunk.Body(self.foot_mass, moment)
        self.footL.position = self.heel_vec + 0.5 * self.foot_vec
        self.heelL_shape = pymunk.Circle(self.footL, self.rad, -0.5 * self.foot_vec)
        self.heelL_shape.impulse = (0,0)
        self.toeL_shape = pymunk.Circle(self.footL, self.rad, 0.5 * self.foot_vec)
        self.toeL_shape.impulse = (0,0)
        self.footR = pymunk.Body(self.foot_mass, moment)
        self.footR.position = self.heel_vec + 0.5 * self.foot_vec
        self.heelR_shape = pymunk.Circle(self.footR, self.rad, -0.5 * self.foot_vec)
        self.heelR_shape.impulse = (0,0)
        self.toeR_shape = pymunk.Circle(self.footR, self.rad, 0.5 * self.foot_vec)
        self.toeR_shape.impulse = (0,0)

        self.space.add(self.torso, self.thighL, self.thighR, self.legL, self.legR, self.footL, self.footR)
        self.space.add(self.heelL_shape, self.toeL_shape, self.heelR_shape, self.toeR_shape)
        self.n_bodies = 7 # number of bodies
        self.n_states = 17 # 9 degrees of freedom + 8 pressure sensors

        # Joint limits from Roaas 1982
        self.hipmin  =  -30.0*np.pi/180.0 # -9.5
        self.hipmax  =  120.0*np.pi/180.0
        self.kneemin = -144.0*np.pi/180.0
        self.kneemax =    1.6*np.pi/180.0
        self.anklemin = -60.0*np.pi/180.0 # -40
        self.anklemax =  30.0*np.pi/180.0 # 15
        
        # Add joints and joint limits
        self.hipL   = pymunk.PivotJoint(self.torso, self.thighL, self.hip_vec)
        self.hipL_limit = pymunk.RotaryLimitJoint(self.torso, self.thighL, self.hipmin, self.hipmax)
        self.hipL_limit.error_bias = 0.0
        self.hipR   = pymunk.PivotJoint(self.torso, self.thighR, self.hip_vec)
        self.hipR_limit = pymunk.RotaryLimitJoint(self.torso, self.thighR, self.hipmin, self.hipmax)
        self.hipR_limit.error_bias = 0.0
        self.kneeL  = pymunk.PivotJoint(self.thighL, self.legL, self.leg_vec)
        self.kneeL_limit = pymunk.RotaryLimitJoint(self.thighL, self.legL, self.kneemin, self.kneemax)
        self.kneeL_limit.error_bias = 0.0
        self.kneeR  = pymunk.PivotJoint(self.thighR, self.legR, self.leg_vec)
        self.kneeR_limit = pymunk.RotaryLimitJoint(self.thighR, self.legR, self.kneemin, self.kneemax)
        self.kneeR_limit.error_bias = 0.0
        self.ankleL = pymunk.PivotJoint(self.legL, self.footL, self.zero_vec)
        self.ankleL_limit = pymunk.RotaryLimitJoint(self.legL, self.footL, self.anklemin, self.anklemax)
        self.ankleL_limit.error_bias = 0.0
        self.ankleR = pymunk.PivotJoint(self.legR, self.footR, self.zero_vec)
        self.ankleR_limit = pymunk.RotaryLimitJoint(self.legR, self.footR, self.anklemin, self.anklemax)
        self.ankleR_limit.error_bias = 0.0
        self.space.add(self.hipL, self.hipR, self.kneeL, self.kneeR, self.ankleL, self.ankleR)
        self.space.add(self.hipL_limit, self.hipR_limit, self.kneeL_limit, self.kneeR_limit)
        self.space.add(self.ankleL_limit, self.ankleR_limit)
        self.n_joints = 6
        
        # Add thick ground so walker won't fall through
        self.space_left   = -1.0
        self.space_right  =  3.0
        self.space_bottom = -0.5
        self.space_top    =  3.0
        
        self.ground = pymunk.Poly(self.space.static_body, 
                                  ((self.space_right, 0.0), (self.space_left, 0.0),
                                   (self.space_left, -1.0), (self.space_right, -1.0)))
        self.space.add(self.ground)

        # Collision properties
        collision_types = {"ground": 0, "foot": 1}
        self.ch_foot = self.space.add_collision_handler(collision_types["foot"], collision_types["ground"])
        self.ch_foot.post_solve = self.get_foot_impulse
        self.ch_foot.separate = self.zero_foot_impulse
        self.ground.friction = self.fric
        self.ground.elasticity = self.elas
        self.ground.collision_type = collision_types["ground"]
        self.heelL_shape.elasticity = self.elas
        self.heelL_shape.friction = self.fric
        self.heelL_shape.filter = pymunk.ShapeFilter(group=1)
        self.heelL_shape.collision_type = collision_types["foot"]
        self.toeL_shape.elasticity = self.elas
        self.toeL_shape.friction = self.fric
        self.toeL_shape.filter = pymunk.ShapeFilter(group=1)
        self.toeL_shape.collision_type = collision_types["foot"]
        self.heelR_shape.elasticity = self.elas
        self.heelR_shape.friction = self.fric
        self.heelR_shape.filter = pymunk.ShapeFilter(group=1)
        self.heelR_shape.collision_type = collision_types["foot"]
        self.toeR_shape.elasticity = self.elas
        self.toeR_shape.friction = self.fric
        self.toeR_shape.filter = pymunk.ShapeFilter(group=1)
        self.toeR_shape.collision_type = collision_types["foot"]
        
        return

    # initialize body position from pose parameters
    # geometry = (torso_length, thigh_length, leg_length, foot_length, foot_pivot, rad)
    # bodies = (torso thighL thighR legL legR footL footR)
    # pose = (llunge rlunge squat lean drop)
    def set_pose(self, pose):
        llunge = pose[0]     # left ankle position as a fraction of total leg length [0,1]
        rlunge = pose[1]     # right ankle position as a fraction of total leg length [0,1]
        squat  = pose[2]     # hip height as a fraction of max, given llunge and rlunge [0,1]
        lean   = pose[3]     # lean of torso to the right with respect to gravity [limited by hip_min and hip_max]
        drop   = pose[4]     # How far above the ground the walker is floating
        
        self.leg = 1 * (rlunge > llunge) # which leg is in front: 0=L 1=R
        self.swapping_stance = False
        
        tor_len = self.torso_length
        thi_len = self.thigh_length
        leg_len = self.leg_length
        fot_len = self.foot_length
        fot_piv = self.foot_pivot
        
        floatY = self.rad + drop    # height of ankle off ground - increase this to drop walker
        po2 = 0.5 * np.pi

        # Ankle positions
        leg_tot = thi_len + leg_len
        lankleX = llunge * leg_tot
        rankleX = rlunge * leg_tot
        lankle_pos = Vec2d(lankleX, floatY)
        rankle_pos = Vec2d(rankleX, floatY)

        # Hip position
        hip_height = squat * math.sqrt(leg_tot**2 - max(lankleX**2,rankleX**2))
        hip_pos = Vec2d(0.0,hip_height + floatY)

        # Knee positions
        lah = math.sqrt(hip_height**2 + lankleX**2)
        lah_leg_angle = math.acos(np.clip((lah**2 + leg_len**2 - thi_len**2) / (2.0 * lah * leg_len),-1.0,1.0))
        lah_angle = math.acos(-lankleX / lah)
        lleg_angle = lah_angle - lah_leg_angle
        lknee_pos = lankle_pos + leg_len * Vec2d(math.cos(lleg_angle), math.sin(lleg_angle))
        rah = math.sqrt(hip_height**2 + rankleX**2)
        rah_leg_angle = math.acos(np.clip((rah**2 + leg_len**2 - thi_len**2) / (2.0 * rah * leg_len),-1.0,1.0))
        rah_angle = math.acos(-rankleX / rah)
        rleg_angle = rah_angle - rah_leg_angle
        rknee_pos = rankle_pos + leg_len * Vec2d(math.cos(rleg_angle), math.sin(rleg_angle))

        # Set body positions, angles, and velocities
        self.torso.position = hip_pos + 0.5 * tor_len * Vec2d(math.sin(lean),math.cos(lean))
        self.torso.angle = -lean
        self.torso.velocity = (0.0,0.0)
        self.torso.angular_velocity = 0.0
        self.thighL.position = 0.5 * (hip_pos + lknee_pos)
        self.thighL.angle = (hip_pos-lknee_pos).angle - po2
        self.thighL.velocity = (0.0,0.0)
        self.thighL.angular_velocity = 0.0
        self.thighR.position = 0.5 * (hip_pos + rknee_pos)
        self.thighR.angle = (hip_pos-rknee_pos).angle - po2
        self.thighR.velocity = (0.0,0.0)
        self.thighR.angular_velocity = 0.0
        self.legL.position = 0.5 * (lknee_pos + lankle_pos)
        self.legL.angle = (lknee_pos-lankle_pos).angle - po2
        self.legL.velocity = (0.0,0.0)
        self.legL.angular_velocity = 0.0
        self.legR.position = 0.5 * (rknee_pos + rankle_pos)
        self.legR.angle = (rknee_pos-rankle_pos).angle - po2
        self.legR.velocity = (0.0,0.0)
        self.legR.angular_velocity = 0.0
        foot_offset = -fot_piv + 0.5*fot_len
        self.footL.position = lankle_pos + Vec2d(foot_offset,0.0)
        self.footL.angle = 0.0
        self.footL.velocity = (0.0,0.0)
        self.footL.angular_velocity = 0.0
        self.footR.position = rankle_pos + Vec2d(foot_offset,0.0)
        self.footR.angle = 0.0
        self.footR.velocity = (0.0,0.0)
        self.footR.angular_velocity = 0.0
        min_thigh_angle = min(self.thighL.angle,self.thighR.angle)
        # lean forward if thigh is too far back
        if min_thigh_angle < self.torso.angle + self.hipmin:
            self.torso.angle = min_thigh_angle - self.hipmin
        return

    # Convert pymunk geometry to state vector
    def get_walker_state(self):
        kneeL_angle = self.thighL.angle - self.legL.angle
        kneeR_angle = self.thighR.angle - self.legR.angle
        ankleL_angle = self.legL.angle - self.footL.angle
        ankleR_angle = self.legR.angle - self.footR.angle
        
        # State: [0-9] torsoX torsoY torsoAngle LthighAngle RthighAngle LkneeAngle RkneeAngle LankleAngle RankleAngle
        #        [10-17] LheelX LheelY LtoeX LtoeY RheelX RheelY RtoeX RtoeY

        self.state = (self.torso.position[0], self.torso.position[1], self.torso.angle,
                 self.thighL.angle, self.thighR.angle,
                 kneeL_angle, kneeR_angle, ankleL_angle, ankleR_angle,
                 self.heelL_shape.impulse[0], self.heelL_shape.impulse[1],
                 self.toeL_shape.impulse[0], self.toeL_shape.impulse[1],
                 self.heelR_shape.impulse[0], self.heelR_shape.impulse[1],
                 self.toeR_shape.impulse[0], self.toeR_shape.impulse[1])
        
        return self.state

    # Convert state vector to approximately ±1 for input to network
    def scale_state(self):
        scaled_state = np.copy(self.state)
        scaled_state[1] -= self.mid_torso
        scaled_state[9:17] *= 0.5
        return scaled_state
    
    # Get CG positions of body parts for CG plot
    def get_body_positions(self):
        self.positions = (self.torso.position, self.thighL.position, self.thighR.position,
                          self.legL.position, self.legR.position, self.footL.position, self.footR.position)
        return self.positions

    # Get foot height
    def foot_height(self,leg):
        if leg==0:
            h = self.footL.position[1]
        else:
            h = self.footR.position[1]
        return h

    # Get total CG
    def get_body_CG(self):
        self.CG = (self.torso_mass * self.torso.position +
              self.thigh_mass * (self.thighL.position + self.thighR.position) +
              self.leg_mass * (self.legL.position + self.legR.position) +
              self.foot_mass * (self.footL.position + self.footR.position) ) / self.total_mass
        self.foot_center = 0.5 * (self.footL.position[0] + self.footR.position[0])
        self.foot_spread = self.footL.position[0] - self.footR.position[0]
        self.CG_offset = self.CG[0] - self.foot_center
        self.stability = 0.5 * abs(self.foot_spread) - abs(self.CG_offset)

        return self.CG

    # Apply torques to joints
    # Positive torques crank things counterclockwise
    # Input torques are normalized and scaled here
    def apply_torques(self,norm_torques):
        # torques: hipL hipR kneeL kneeR ankleL ankleR
        t_hipL   = norm_torques[0] * self.torque_max_hip
        t_hipR   = norm_torques[1] * self.torque_max_hip
        t_kneeL  = norm_torques[2] * self.torque_max_knee
        t_kneeR  = norm_torques[3] * self.torque_max_knee
        t_ankleL = norm_torques[4] * self.torque_max_ankle
        t_ankleR = norm_torques[5] * self.torque_max_ankle
        self.torso.torque += t_hipL + t_hipR
        self.thighL.torque = self.thighL.torque - t_hipL + t_kneeL
        self.thighR.torque = self.thighR.torque - t_hipR + t_kneeR
        self.legL.torque = self.legL.torque - t_kneeL + t_ankleL
        self.legR.torque = self.legR.torque - t_kneeR + t_ankleR
        self.footL.torque = self.footL.torque - t_ankleL
        self.footR.torque = self.footR.torque - t_ankleR
        return

    # Callback to get force on foot sensor when foot collides with ground
    def get_foot_impulse(self,arbiter, space, data):
        foot_circle,b = arbiter.shapes
        foot_circle.impulse = arbiter.total_impulse
        return
    
    # Callback to zero out foot force when foot looses contact with ground
    def zero_foot_impulse(self,arbiter, space, data):
        foot_circle,b = arbiter.shapes
        foot_circle.impulse = (0,0)
        return
    
    def setup_figure(self):
        fig, ax = plt.subplots(figsize=(6,5))
        ax.set_xlim(self.space_left,self.space_right)
        ax.set_ylim(self.space_bottom,self.space_top)
        ax.plot((self.space_left,self.space_right),(0,0))
        return fig,ax

    def animate_walker(self,fig,ax,state_record,position_record,CGdots=True):
        # Convert state to endpoints
        # geometry = (torso_length, thigh_length, leg_length, foot_length, foot_pivot, rad)
        #     state = (hip_positionX, hip_positionY, torso_angle, thighL_angle, thighR_angle,
        #         kneeL_angle, kneeR_angle, ankleL_angle, ankleR_angle)
        tor_len = self.torso_length
        thi_len = self.thigh_length
        leg_len = self.leg_length
        fot_len = self.foot_length
        fot_piv = self.foot_pivot

        hipX = state_record[:,0] + 0.5 * tor_len * np.sin(state_record[:,2]) 
        hipY = state_record[:,1] - 0.5 * tor_len * np.cos(state_record[:,2])
        headX = hipX - tor_len * np.sin(state_record[:,2])
        headY = hipY + tor_len * np.cos(state_record[:,2])
        kneeLX = hipX + thi_len * np.sin(state_record[:,3])
        kneeLY = hipY - thi_len * np.cos(state_record[:,3])
        kneeRX = hipX + thi_len * np.sin(state_record[:,4])
        kneeRY = hipY - thi_len * np.cos(state_record[:,4])
        legL_angle = state_record[:,3] - state_record[:,5]
        ankleLX = kneeLX + leg_len * np.sin(legL_angle)
        ankleLY = kneeLY - leg_len * np.cos(legL_angle)
        legR_angle = state_record[:,4] - state_record[:,6]
        ankleRX = kneeRX + leg_len * np.sin(legR_angle)
        ankleRY = kneeRY - leg_len * np.cos(legR_angle)
        footL_angle = legL_angle - state_record[:,7]
        heelLX = ankleLX - fot_piv * np.cos(footL_angle)
        heelLY = ankleLY - fot_piv * np.sin(footL_angle)
        footR_angle = legR_angle - state_record[:,8]
        heelRX = ankleRX - fot_piv * np.cos(footR_angle)
        heelRY = ankleRY - fot_piv * np.sin(footR_angle)
        toeLX = heelLX + fot_len * np.cos(footL_angle)
        toeLY = heelLY + fot_len * np.sin(footL_angle)
        toeRX = heelRX + fot_len * np.cos(footR_angle)
        toeRY = heelRY + fot_len * np.sin(footR_angle)

        # make lines
        torso_line, = ax.plot((hipX[0],headX[0]),(hipY[0],headY[0]))
        thighL_line, = ax.plot((hipX[0],kneeLX[0]),(hipY[0],kneeLY[0]))
        thighR_line, = ax.plot((hipX[0],kneeRX[0]),(hipY[0],kneeRY[0]))
        legL_line, = ax.plot((kneeLX[0],ankleLX[0]),(kneeLY[0],ankleLY[0]))
        legR_line, = ax.plot((kneeRX[0],ankleRX[0]),(kneeRY[0],ankleRY[0]))
        footL_line, = ax.plot((heelLX[0],toeLX[0]),(heelLY[0],toeLY[0]))
        footR_line, = ax.plot((heelRX[0],toeRX[0]),(heelRY[0],toeRY[0]))

        if CGdots:
            CG, = ax.plot(position_record[0,:,0],position_record[0,:,1],'o')

        def animate(i):
            torso_line.set_xdata((hipX[i],headX[i]))
            torso_line.set_ydata((hipY[i],headY[i]))
            thighL_line.set_xdata((hipX[i],kneeLX[i]))
            thighL_line.set_ydata((hipY[i],kneeLY[i]))
            thighR_line.set_xdata((hipX[i],kneeRX[i]))
            thighR_line.set_ydata((hipY[i],kneeRY[i]))
            legL_line.set_xdata((kneeLX[i],ankleLX[i]))
            legL_line.set_ydata((kneeLY[i],ankleLY[i]))
            legR_line.set_xdata((kneeRX[i],ankleRX[i]))
            legR_line.set_ydata((kneeRY[i],ankleRY[i]))
            footL_line.set_xdata((heelLX[i],toeLX[i]))
            footL_line.set_ydata((heelLY[i],toeLY[i]))
            footR_line.set_xdata((heelRX[i],toeRX[i]))
            footR_line.set_ydata((heelRY[i],toeRY[i]))
            parts=(torso_line, thighL_line, thighR_line, legL_line, legR_line, footL_line, footR_line)
            if CGdots:
                CG.set_xdata(position_record[i,:,0])
                CG.set_ydata(position_record[i,:,1])
                parts = (parts, CG)

            return parts

        ani = animation.FuncAnimation(fig, animate, interval=10, frames=nt, blit=True, save_count=50)

        return ani

# Teaching Modules:
# Input(self.state): [0-8] torsoX torsoY torsoAngle LthighAngle RthighAngle LkneeAngle RkneeAngle LankleAngle RankleAngle
#        [9-16] LheelX LheelY LtoeX LtoeY RheelX RheelY RtoeX RtoeY
# Output: [0-5] torques - hipL hipR kneeL kneeR ankleL ankleR
#   hips:    positive extends thigh     (+:glutes -:psoas)
#   knees:   positive flexes knee       (+:hamstrings -:quads)
#   ankles:  positive plantarflexes foot  (+:gastroc -:tibialis_anterior)
# positive angle is counter-clockwise

# w.get_body_CG() calculates w.CG, w.foot_spread (L-R), and w.CG_offset, and sets w.stability

# stand with bent knees, ready to lean forward
    # 'leg' refers to forward leg: L=0 R=1
    def backflip(self):
        torques = np.zeros(6)
        s = self.state
        
        deg2rad = np.pi / 180.
        squatting_torso_angle = 45 * deg2rad
        squatting_knee_bend = 90 * deg2rad
        landing_thigh_angle = 10 * deg2rad
        landing_knee_bend = 40 * deg2rad

        torso_gain = 0.25
        hip_gain = 0.5
        torso_throwback_torque = 3.7
        torso_tuck_torque = -0.2
        quad_jump_torque = -0.41
        calf_jump_torque =  0.5
        calf_land_torque = -0.05
        catching_fall_knee_bend = np.pi/2.
        catching_fall_knee_bend_time = 0.05
        
        if self.phase == 0: # squatting
            if (s[5] < squatting_knee_bend):   # need to squat more
                # net torque on torso (=torque[0]+torque[1])
                torso_torque = -torso_gain * (s[2] + squatting_torso_angle)
                torques[0] = torso_torque
                torques[1] = torso_torque

                # set knee torques
                torques[2] = self.knee_torque(0,squatting_knee_bend,0.3)
                torques[3] = torques[2]
            else:  # enough squatting, time to jump
                self.phase = 1 # jumping
                
        if self.phase == 1: # jumping
            if (s[10]>0 or s[12]>0 or s[14]>0 or s[16]>0):   # feet still on ground
                # net torque on torso (=torque[0]+torque[1])
                torques[0] = torso_throwback_torque
                torques[1] = torso_throwback_torque

                # set knee torques
                torques[2] = quad_jump_torque
                torques[3] = quad_jump_torque
                torques[4] = calf_jump_torque
                torques[5] = calf_jump_torque

            else:  # feet left ground, time to go into flying mode
                self.phase = 2 # flying
                
        if self.phase == 2: # flying
#            if not (s[10]>0 or s[12]>0 or s[14]>0 or s[16]>0):   # still flying
            if self.torso.angle < np.pi:   # torso not yet upside down
                torques[0] = torso_tuck_torque
                torques[1] = torso_tuck_torque - 0.1
                torques[4] = calf_land_torque
                torques[5] = calf_land_torque
            else:# past upside down, time to prepare to land
                self.phase = 3 # landing
        
        if self.phase == 3: # landing
            torques[0] = torso_gain * ((s[3]-s[2]) - landing_thigh_angle)
            torques[1] = torso_gain * ((s[4]-s[2]) + landing_thigh_angle) - 0.5
            torques[2] = self.knee_torque(0,landing_knee_bend,2.0)
            torques[3] = self.knee_torque(1,landing_knee_bend,2.0)
            torques[4] = calf_land_torque
            torques[5] = calf_land_torque        

        return torques
    
    # Knee torques to keep legs close to target_bend
    def knee_torque(self,leg,target_bend,support_factor):
        s = self.state

        # stiffen knees at target_bend
        # Support torque = M G L sin(bend/2)
        # MGLN is normalized to torque_max_knee
        bend = s[5+leg]
        support_torque = self.MGLN * np.sin(0.5*bend)
        extend_angle_gain = 0.1
        flex_angle_gain = 0.1

        if leg==0:
            bend_rate = self.thighL.angular_velocity - self.legL.angular_velocity
        else:
            bend_rate = self.thighR.angular_velocity - self.legR.angular_velocity
        
        touching_ground = (s[10+4*leg] or s[12+4*leg])
        if touching_ground:
            if bend_rate > 0:
                viscosity_gain = 0.05
            else:
                viscosity_gain = 0.2
        else:
            viscosity_gain = 0.05

        tq = -np.clip(support_factor * support_torque * touching_ground
                      + (bend>target_bend) * extend_angle_gain * (bend-target_bend)
                      + (bend<target_bend) *   flex_angle_gain * (bend-target_bend)
                      + np.sign(bend_rate) * (viscosity_gain * bend_rate)**2, -1., 1.)
                
        return tq



# Go

In [3]:
def stepfunc(a,thresh):
    return (a>thresh).astype(nodetype)

np.random.seed(1)
nodetype = 'int16'

# Make Walker
w = Walker()

# Initialize network
n_nodes = 10000
n_active = 20
n_keys = 4 # 0:squat 1:jump 2:fly 3:land
keys = np.concatenate((np.ones((n_keys,n_active),dtype=nodetype),np.zeros((n_keys,n_nodes-n_active),dtype=nodetype)),axis=1)
for i in range(n_keys):
    np.random.shuffle(keys[i,:])

# reservoir network to set time base
weight_density = 0.1
weight_mean = 0.0
weight_std = 0.1
activation_threshold = 1.0
weights = sw.sparse_weights(n_nodes,density=weight_density,weight_std=weight_std)

# output network to learn behavior
n_out = 6  # number of joints to control
output_weights = np.zeros((n_out,n_nodes))
output_learning_rate = 0.045

ntrials = 2

time = 1.25  # 2.0
dt = 0.01
nt = int(time / dt)
w.dt = dt

w.state_record = np.zeros((ntrials,nt,w.n_states))
position_record = np.zeros((ntrials,nt,w.n_bodies,2))
activity_record = np.zeros((nt,n_nodes))
output_activity_record = np.zeros((nt,ntrials,n_out))
phase_record = -np.ones((ntrials,nt))

deg2rad = np.pi / 180.
leg_angle = 5 * deg2rad # 0.25
llunge = np.sin(leg_angle)
rlunge = llunge
squat = 0.98
forward_lean = 1 * deg2rad
drop = 0.01 # 0.01
starting_pose   = (llunge, rlunge, squat, forward_lean, drop)

for trial in range(ntrials):
        
    w.leg = 0
    # leg refers to the leg that is in FRONT
    
    w.set_pose(starting_pose)
    w.state_record[trial,0,:] = w.get_walker_state()
    position_record[trial,0,:,:] = w.get_body_positions()
        
    w.phase = 0 # squatting
    
    # restart nodes
    nodes = keys[0,:].copy()
    for t in range(1,nt):
        activity_record[t,:] = nodes
        
        # Time step for walker
        w.space.step(dt)
        w.state_record[trial,t,:] = w.get_walker_state()
        position_record[trial,t,:,:] = w.get_body_positions()
        phase_record[trial,t] = w.phase
        
        # Run teacher to set lesson and calculate phase
        w.get_body_CG() # calculates w.CG, w.foot_spread, w.CG_offset, and w.stability
        scaled_state = w.scale_state()
        lesson = w.backflip()
        phase_record[trial,t] = w.phase

        if trial%2 == 0: # train
            output = lesson.copy()
            output_weights += output_learning_rate * np.outer(lesson,nodes)
        else:            # test
            output = output_weights.dot(nodes)

        output_activity_record[t,trial,:] = output
        w.apply_torques(output)

        # Advance reservoir
#        nodes = stepfunc(weights.dot(nodes),activation_threshold)
        sorted_indices = np.argsort(weights.dot(nodes))
        nodes[sorted_indices[0:n_nodes-n_active]] = 0
        nodes[sorted_indices[n_nodes-n_active:n_nodes]] = 1
    
fig,ax = plt.subplots(2, 2, figsize=(8,8))
im=ax[0][0].imshow(output_weights,aspect=n_nodes/n_out)
fig.colorbar(im,ax=ax[0][0])
im=ax[0][1].imshow(activity_record,aspect=n_nodes/nt)
fig.colorbar(im,ax=ax[0][1])
im=ax[1][0].imshow(np.reshape(output_activity_record,(nt,-1)),aspect=ntrials*n_out/nt)
fig.colorbar(im,ax=ax[1][0])
cormat = activity_record @ activity_record.T
#im=ax[1][1].imshow(cormat)
im=ax[1][1].imshow(phase_record.T,aspect=ntrials/nt)
fig.colorbar(im,ax=ax[1][1])
plt.show()

FigureCanvasNbAgg()

# Animate Teacher

In [4]:
afig,ax = w.setup_figure()
trial = 0
ani = w.animate_walker(afig,ax,w.state_record[trial,:,:],position_record[trial,:,:,:],CGdots=False)
ani.event_source.start()

FigureCanvasNbAgg()

In [5]:
ani.event_source.stop()
plt.close(afig)

In [6]:
ani.save("movie_backflip_teacher.mp4")

# Animate Learner

In [7]:
afig,ax = w.setup_figure()
trial = 1
ani = w.animate_walker(afig,ax,w.state_record[trial,:,:],position_record[trial,:,:,:],CGdots=False)
ani.event_source.start()

FigureCanvasNbAgg()

In [8]:
ani.event_source.stop()
plt.close(afig)

In [9]:
ani.save("movie_backflip_learner.mp4")

In [10]:
fig, ax = plt.subplots(1, 3, figsize=(10,4))
trial = 0
ax[0].plot(position_record[trial,:,:,0],position_record[trial,:,:,1])
ax[1].plot(w.state_record[trial,:,9::2])
ax[2].plot(w.state_record[trial,:,10::2])
plt.show()

FigureCanvasNbAgg()